# Univariate Models

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src.data_preprocessing.data_pipeline import data_pipeline
from src.modeling.univariate_modeling import get_arima_model
from src.modeling.evaluation import smape, mae

In [ ]:
DATA_DIR = (
    Path("..")
    / ".."
    / ".."
    / "hfactory_magic_folders"
    / "plastic_cost_prediction"
    / "data"
)
MAIN_FILE = "PA6_cleaned_dataset.csv"

In [ ]:
df, _ = data_pipeline(DATA_DIR / MAIN_FILE)

In [ ]:
models = get_arima_model(df)

In [ ]:
for col, model in models.items():
    d = model.model.order[1]
    y_true = df[col].dropna().iloc[d:]
    time = y_true.index
    y_pred = model.fittedvalues.iloc[d:]

    print(f"*** {col} ***")
    print(f"sMAPE: {smape(y_true.values, y_pred.values):.2f}%")
    print(f"MAE: {mae(y_true.values, y_pred.values):.2f}")

    # model summary
    print(model.summary())

    # plotting the original series and the fitted values
    plt.figure(figsize=(10, 6))
    plt.plot(time, y_true, label="Actual Series")
    plt.plot(time, y_pred, label="Fitted Values", linestyle="dashed")
    plt.xlabel("Time")
    plt.ylabel("Values")
    plt.legend()
    plt.show()

    # line plot of residuals
    residuals = pd.DataFrame(model.resid)
    residuals.plot()
    plt.title("Residuals")
    plt.show()
    print("--------------------------------")